In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Define constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 2  # Use only 2 classes
EPOCHS = 2
SAMPLES_PER_CLASS = 10

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convert labels to integers for easy indexing
y_train = y_train.flatten()
y_test = y_test.flatten()

# Select only classes 0 and 1 (airplane and automobile)
classes_to_use = [0, 1]
train_indices = np.isin(y_train, classes_to_use)
test_indices = np.isin(y_test, classes_to_use)

x_train_filtered = x_train[train_indices]
y_train_filtered = y_train[train_indices]
x_test_filtered = x_test[test_indices]
y_test_filtered = y_test[test_indices]

# Normalize pixel values to [0, 1]
x_train_filtered = x_train_filtered / 255.0
x_test_filtered = x_test_filtered / 255.0

# Convert labels to categorical
y_train_filtered = to_categorical(y_train_filtered, NUM_CLASSES)
y_test_filtered = to_categorical(y_test_filtered, NUM_CLASSES)

# Function to downsample data
def downsample_to_n_samples(x, y, n):
    x_new = []
    y_new = []
    for i in range(NUM_CLASSES):
        class_indices = np.where(np.argmax(y, axis=1) == i)[0]
        selected_indices = np.random.choice(class_indices, size=n, replace=False)
        x_new.append(x[selected_indices])
        y_new.append(y[selected_indices])
    return np.concatenate(x_new, axis=0), np.concatenate(y_new, axis=0)

# Downsample training and test data
x_train_reduced, y_train_reduced = downsample_to_n_samples(x_train_filtered, y_train_filtered, SAMPLES_PER_CLASS)
x_test_reduced, y_test_reduced = downsample_to_n_samples(x_test_filtered, y_test_filtered, SAMPLES_PER_CLASS)

# Convert numpy arrays to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_reduced, y_train_reduced))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_reduced, y_test_reduced))

# Function to preprocess and resize images
def preprocess_and_resize(image, label):
    image = tf.image.resize(image, IMAGE_SIZE)
    return image, label

# Apply preprocessing and batching
train_dataset = train_dataset.map(preprocess_and_resize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

test_dataset = test_dataset.map(preprocess_and_resize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

# Load the pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the VGG16 base model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduce dimensionality
x = Dense(1024, activation='relu')(x)  # Add a dense layer
predictions = Dense(NUM_CLASSES, activation='softmax')(x)  # Final classification layer

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=EPOCHS, validation_data=test_dataset)

# Save the model
model.save('transfer_learning_model.h5')

print("Training completed and model saved.")


Epoch 1/2
1/1 ━━━━━━━━━━━━━━━━━━━━ 35s 35s/step - accuracy: 0.5000 - loss: 0.6817 - val_accuracy: 0.5000 - val_loss: 0.6868
Epoch 2/2
1/1 ━━━━━━━━━━━━━━━━━━━━ 32s 32s/step - accuracy: 0.6500 - loss: 0.6670 - val_accuracy: 0.6000 - val_loss: 0.6818


Training completed and model saved.


In [6]:
from google.colab import files

# Upload the image
uploaded = files.upload()

# List uploaded files
for filename in uploaded.keys():
    print(f'User uploaded file "{filename}"')


Saving Tarom.b737-700.yr-bgg.arp.jpg to Tarom.b737-700.yr-bgg.arp.jpg
User uploaded file "Tarom.b737-700.yr-bgg.arp.jpg"


In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image
import io

# Define constants
IMAGE_SIZE = (224, 224)
CLASS_NAMES = ['airplane', 'automobile']  # Update with your classes

# Load the trained model
model = tf.keras.models.load_model('transfer_learning_model.h5')

# Function to preprocess the uploaded image
def load_and_preprocess_image(img_path):
    img = Image.open(img_path)
    img = img.resize(IMAGE_SIZE)  # Resize image
    img_array = image.img_to_array(img)  # Convert to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize to [0, 1]
    return img_array

# Use the filename from the uploaded file
uploaded_file_path = list(uploaded.keys())[0]  # Get the filename of the uploaded image

# Preprocess the image
img_array = load_and_preprocess_image(uploaded_file_path)

# Make prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)

# Print the result
print(f"Predicted class: {CLASS_NAMES[predicted_class[0]]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted class: airplane
